In [60]:
import openai as oai
import os
import pandas as pd
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import Hastebin
import requests
load_dotenv()
oai.api_key  = os.getenv("OPENAI_API_KEY")

In [28]:
scope = "playlist-read-private"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

playlists = sp.current_user_playlists()

songlist = []
for playlist in playlists['items']:
    if playlist['name'] == 'spotAIfy project':
        results = sp.playlist_tracks(playlist['id'])
        for idx, item in enumerate(results['items']):
            track = item['track']
            song = track['artists'][0]['name'] + " – " + track['name']
            songlist.append(song)
print(songlist)


['Taylor Swift – Wildest Dreams', 'Joji – Sanctuary', 'Post Malone – Otherside', 'tobi lou – hot tub DREAM Machine', "Kid Cudi – Elsie's Baby Boy (flashback)"]


In [43]:
chat_prompt = "Make me a new 5 song spotify playlist based on these 5 songs: \n"
for song in songlist:
    chat_prompt += song + "\n"
chat_prompt += "After listing the new songs, write 'End of List'. After the list is completed, give reasoning for each new song. \n"
print(chat_prompt)

Make me a new 5 song spotify playlist based on these 5 songs: 
Taylor Swift – Wildest Dreams
Joji – Sanctuary
Post Malone – Otherside
tobi lou – hot tub DREAM Machine
Kid Cudi – Elsie's Baby Boy (flashback)
After listing the new songs, write 'End of List'. After the list is completed, give reasoning for each new song. 



In [44]:
response = oai.Completion.create(model="text-davinci-003", prompt=chat_prompt, temperature=1, max_tokens = 1000)


Alicia Keys - Underdog
Khalid - Silence
Billie Eilish - Everything I Wanted
Bea Miller - to the grave
Lana Del Rey - Starry Eyed
End of List

Alicia Keys - Underdog: This upbeat anthem is a perfect way to continue the vibe from the previous songs. It's a great way to keep the party going and keep the energy up. 

Khalid - Silence: For a more laid-back and reflective track, Khalid's Silence is the way to go. It's a great contrast after the upbeat vibes, and also fits with the overall mellow theme of the list.

Billie Eilish - Everything I Wanted: This track is an upbeat and fast-paced pop song that adds to the mix and keeps the energy going. Its catchy melody will keep the listener engaged. 

Bea Miller - to the grave: For a mellow and laid-back track, Bea Miller's to the grave is a great option. It's a slower paced track that will bring a more relaxed atmosphere. 

Lana Del Rey - Starry Eyed: This is a perfect way to end the list as it has a dreamy vibe that ties in with the title of 

In [45]:
new_playlist_response = response['choices'][0]['text']
print(new_playlist_response)


Alicia Keys - Underdog
Khalid - Silence
Billie Eilish - Everything I Wanted
Bea Miller - to the grave
Lana Del Rey - Starry Eyed
End of List

Alicia Keys - Underdog: This upbeat anthem is a perfect way to continue the vibe from the previous songs. It's a great way to keep the party going and keep the energy up. 

Khalid - Silence: For a more laid-back and reflective track, Khalid's Silence is the way to go. It's a great contrast after the upbeat vibes, and also fits with the overall mellow theme of the list.

Billie Eilish - Everything I Wanted: This track is an upbeat and fast-paced pop song that adds to the mix and keeps the energy going. Its catchy melody will keep the listener engaged. 

Bea Miller - to the grave: For a mellow and laid-back track, Bea Miller's to the grave is a great option. It's a slower paced track that will bring a more relaxed atmosphere. 

Lana Del Rey - Starry Eyed: This is a perfect way to end the list as it has a dreamy vibe that ties in with the title of 

In [62]:
new_playlist_songs = list(filter(None, new_playlist_response.split('End of List')[0].split('\n')))
songs = {}

# Loop through the lines and split each one into a song name and artist
for line in new_playlist_songs:
    artist, song = line.split(' - ')
    songs[song] = artist
    
new_playlist_descriptions = list(filter(None, new_playlist_response.split('End of List')[1].split('\n')))
songs

{'Underdog': 'Alicia Keys',
 'Silence': 'Khalid',
 'Everything I Wanted': 'Billie Eilish',
 'to the grave': 'Bea Miller',
 'Starry Eyed': 'Lana Del Rey'}

In [64]:
track_ids = {}

# Loop through each song in the 'songs' dictionary
for song in songs:
    # Search for the song using the Spotipy library
    results = sp.search(q=song, type='track', limit=1)
    
    # Extract the track ID from the search results
    if len(results['tracks']['items']) > 0:
        track_id = results['tracks']['items'][0]['id']
        track_ids[song] = track_id
    else:
        print(f"No track found for {song}")
    
# Print the resulting dictionary of song names and track IDs
print(track_ids)

{'Underdog': '7FsAFw1oZeTJUZmIzSh0cg', 'Silence': '7vGuf3Y35N4wmASOKLUVVU', 'Everything I Wanted': '3ZCTVFBt2Brf31RLEnCkWJ', 'to the grave': '16itsMCWyAoz1rOVbWQUXI', 'Starry Eyed': '6A4Kuy7JL0Znab3Skgloiv'}


In [61]:
text = new_playlist_response.split('End of List')[1]
url = "https://hastebin.com/documents"
response = requests.post(url, data=text.encode('utf-8'))

if response.status_code == 200:
    key = response.json()["key"]
    print(f"Hastebin URL: https://hastebin.com/{key}")
else:
    print("Failed to create hastebin")


Failed to create hastebin


In [67]:
playlist_name = "spotAIfy generated"
scope = "playlist-modify-private"
new_playlist = sp.user_playlist_create(user=sp.current_user()['id'], name=playlist_name)
playlist_id = playlist['id']
sp.user_playlist_add_tracks(sp.current_user()['id'], playlist_id, track_ids)



# Add the songs to the new playlist
sp.playlist_add_items(playlist_id=new_playlist['id'], items=songs)

HTTP Error for POST to https://api.spotify.com/v1/users/dhruvgandhi853/playlists with Params: {} returned 403 due to Insufficient client scope


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/users/dhruvgandhi853/playlists:
 Insufficient client scope, reason: None